In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap

In [2]:
import PIL
print('PIL',PIL.__version__)
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

PIL 8.2.0


In [14]:
parent_directory = "C:\chip_allocation_temp"
annotation_directory = "Sunny"
number_of_tiles = 20
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"

In [24]:
dist = ap.annotator(annotation_directory) #create the processing class
dist.state_dcc_directory(parent_directory)
dist.number_of_tiles(number_of_tiles)
dist.get_tile_urls(tiles_remaining)
dist.make_subdirectories()

Unlabeled Tiles (1106, 2)


In [22]:
row_to_remove = 8
error_downloading = np.concatenate((np.load('tile_name_tile_url_error_downloading.npy'), dist.tile_name_tile_url_tiles_for_annotators[row_to_remove]))
np.save('tile_name_tile_url_error_downloading.npy', error_downloading)

In [23]:
# Remove the rows with errors
np.load('tile_name_tile_url_remaining_expanded.npy').shape
np.save('tile_name_tile_url_remaining_expanded',np.delete(np.load(tiles_remaining), row_to_remove, axis = 0))
np.load('tile_name_tile_url_remaining_expanded.npy').shape

#recover the unlabeled images from the dist
## np.save('tile_name_tile_url_remaining_expanded',dist.tile_name_tile_url_unlabeled)
#dist.tile_name_tile_url_tiles_for_annotators

(1106, 2)

In [25]:
dist.download_images()
dist.tile_rename()

Bypassing download of already-downloaded file m_3309805_se_14_060_20181027.tif
Bypassing download of already-downloaded file m_3010201_nw_13_060_20181029_20190503.tif
Bypassing download of already-downloaded file m_3210254_sw_13_060_20181026.tif
Bypassing download of already-downloaded file m_3209426_sw_15_060_20181120_20190222.tif
Bypassing download of already-downloaded file m_3508408_ne_16_060_20181024.tif
Bypassing download of already-downloaded file m_3110615_sw_13_060_20190126_20190504.tif
Bypassing download of already-downloaded file m_3009531_nw_15_060_20181126_20190503.tif
Bypassing download of already-downloaded file m_4109752_se_14_060_20180709_20181210.tif


100% (511992171 of 511992171) |##########| Elapsed Time: 0:00:16 Time:  0:00:16


...done, 511992171 bytes.


100% (557620388 of 557620388) |##########| Elapsed Time: 0:00:14 Time:  0:00:14


...done, 557620388 bytes.


...

100% (573955570 of 573955570) |##########| Elapsed Time: 0:00:16 Time:  0:00:16


...done, 573955570 bytes.


100% (447335902 of 447335902) |##########| Elapsed Time: 0:00:14 Time:  0:00:14


...done, 447335902 bytes.
['https_naipblobs.blob.core.windows.net_naip_v002_ca_2018_ca_060cm_2018_34117_m_3411730_nw_11_060_20180805_20190209.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_ca_2018_ca_060cm_2018_34119_m_3411956_nw_11_060_20180724_20190209.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_ca_2018_ca_060cm_2018_35119_m_3511935_se_11_060_20180714_20190208.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_ca_2018_ca_060cm_2018_38121_m_3812130_nw_10_060_20180714_20190210.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_in_2018_in_060cm_2018_40086_m_4008661_sw_16_060_20180903_20181211.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_in_2018_in_060cm_2018_41087_m_4108720_se_16_060_20180707_20190221.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_ma_2018_ma_060cm_2018_42070_m_4207041_sw_19_060_20181107.tif', 'https_naipblobs.blob.core.windows.net_naip_v002_mi_2018_mi_060cm_2018_42084_m_4208445_ne_16_060_20180707_20181211.tif', 'https_naipblo

IndexError: list index out of range

In [ ]:
dist.chip_tiles()

In [4]:
tile_names = os.listdir(dist.tiles_dir) #get a list of all of the tiles in tiles directory

In [1]:
for tile_name in tile_names: #index over the tiles in the tiles_dir 

    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)
    tile = Image.open(os.path.join(dist.tiles_dir, tile_name)) #open image
    tile = tile.convert('RGB') #to file issues

NameError: name 'tile_names' is not defined

In [ ]:
        
for tile_name in tile_names: #index over the tiles in the tiles_dir 

    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)

    tile = Image.open(os.path.join(self.tiles_dir, tile_name)) #open image
    tile = tile.convert('RGB') #to file issues

    width, height = tile.size
    item_width = int(512)
    col = int(width/512)+1
    row = int(height/512)+1
    box_list = []

    count = 1
    for j in range(0,col):
        for i in range(0,row):
            box = (i*item_width, j*item_width, (i+1)*item_width, (j+1)*item_width)

            chips_save_path = os.path.join(self.chips_dir, file_name+ '_'+ \
                                           str(count).zfill(6) + '.jpg') 
                # The index is a six-digit number like '000023'. 

            #chip = tile.crop(box)
            #chip = chip.save(chips_save_path)
            chip = tile.crop(box).save(chips_save_path)
            count += 1
    print(count)

In [20]:
dist.track_tile_annotations(tiles_labeled)
np.save('tile_name_tile_url_remaining_expanded', dist.tile_name_tile_url_remaining)
np.save('tile_name_tile_url_labeled', dist.tile_name_tile_url_labeled)